In [83]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 1. Loading dataset

The dataset used for this project come from: https://www.kaggle.com/competitions/digit-recognizer

In [84]:
data_pd = pd.read_csv("train.csv")

**1.1 Inspecting the dataset**

In [85]:
data_pd.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**1.2 Converting Data to NumPy Arrays**

In [86]:
data = np.array(data_pd)
np.random.shuffle(data)
data

array([[3, 0, 0, ..., 0, 0, 0],
       [9, 0, 0, ..., 0, 0, 0],
       [7, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [87]:
m, n = data.shape

**1.3 Splitting dataset**

In [89]:
data_dev = data[:1000].T
X_dev = data_dev[1:]
y_dev = data_dev[0] 

data_train = data[1000:].T
X_train = data_train[1:]
y_train = data_train[0]

### 2. Preparing the neural network

**2.1 Initialize parametrs**

In [ ]:
def init_params():
    W1 = np.random.randn(10, 784)
    b1 = np.random.randn(10, 1)
    W2 = np.random.randn(10, 10)
    b2 = np.random.randn(10, 1)
    return W1, b1, W2, b2

**2.2 Defininig acivation functions**

In [97]:
def ReLU(Z):
    return np.maximum(0, Z)

def softmax(Z):
    Z = Z - np.max(Z, axis=0, keepdims=True) 
    expZ = np.exp(Z)
    return expZ / np.sum(expZ, axis=0, keepdims=True)

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def deriv_ReLU(Z):
    return (Z > 0).astype(float)

**2.3 Defining for and back propagation function**

In [98]:
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def back_prop(Z1, A1, Z2, A2, W2, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)

    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)

    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)
    
    return dW1, db1, dW2, db2


**2.4 Function for predictions and accurancy**

In [99]:
def get_predictions(A2):
    return np.argmax(A2, axis=0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size


**2.5 Updating params and use gradient descent**

In [ ]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2


def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 50 == 0:
            predictions = get_predictions(A2)
            acc = get_accuracy(predictions, Y)
            print(f"Iteration {i} - Accuracy: {acc:.4f}")
    return W1, b1, W2, b2

**2.8 Fitting the neural network**

In [106]:
W1, b1, W2, b2 = gradient_descent(X_train, y_train, 500, 0.1)

Iteration 0 - Accuracy: 0.1075
Iteration 50 - Accuracy: 0.0995
Iteration 100 - Accuracy: 0.1112
Iteration 150 - Accuracy: 0.1112
Iteration 200 - Accuracy: 0.1112
Iteration 250 - Accuracy: 0.1112
Iteration 300 - Accuracy: 0.1112
Iteration 350 - Accuracy: 0.1112
Iteration 400 - Accuracy: 0.1112
Iteration 450 - Accuracy: 0.1112


### 3. Making Predictions

In [107]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_predictions(index, W1, b1, W2, b2):
    # Grab the image for prediction
    current_image = X_train[:, index, None]
    
    # Make prediction
    prediction = make_predictions(current_image, W1, b1, W2, b2)
    
    # Get true label
    label = y_train[index]
    
    print(f"Prediction: {prediction[0]}")
    print(f"Label: {label}")
    
    # Reshape and display image
    image_to_show = current_image[:, 0].reshape((28,28)) * 255
    plt.gray()
    plt.imshow(image_to_show, interpolation="nearest")
    plt.show()


In [108]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, y_dev)

np.float64(0.125)